In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

LookupError: unknown encoding: us_utf_8

In [2]:
def theurl(name):
 answer={"2012":"http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H155", "2013":"http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H163", "population": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H157", "medical": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H154", "risk": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H140", "employment": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H131", "jobs": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H158", "person_round_plan": "http://meps.ahrq.gov/mepsweb/data_stats/download_data_files_codebook.jsp?PUFId=H153"}
 return answer[name]

In [3]:
def make_soup(url):
 response=requests.get(url)
 soup=BeautifulSoup(response.content)
 return soup

In [4]:
this=make_soup(theurl("2013"))

/usr/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html.parser")

  markup_type=markup_type))


Querying this parser traverses the tree:

And it looks like the variable names and column limits are all contained in bits of the code that look like:

    <font class="smallBlack" face="Arial">CANCER DIAGNOSED - SKIN-NONMELANO (&gt;17)</font>,
    <font class="smallBlack" face="Arial">363   </font>,
    <font class="smallBlack" face="Arial">364   </font>,

The next function extracts all the tags in the parse tree with attribute "font" and collects them in a list, that I (in very poor taste) called final. In my defense, the list is really nothing, and nothing is ever final, so there you go.

I then use a helper function I called "prune_list" to get rid of the crud I didn't want to deal with that also happened to be part of a "font" tag. Finally, this function writes the code dictionary table to the file provided as the second argument to the function.

In [13]:
def get_data(soup,target):
 result=[]
 for row in soup.find_all("font"):
  result.append(row.contents)
 final=[(term[0].replace(u'\xa0',u'')).replace(',','') for term in result]
# get rid of junk
 final=prune_list(final,"::")
 final=prune_list(final,"MEPS HC-155")
 final=prune_list(final,"MEPS HC-163")
 final=prune_list(final,"MEPS HC-157")
 final=prune_list(final,"MEPS H154 CODEBOOK")
 final=prune_list(final,"MEPS H140 CODEBOOK")
 final=prune_list(final,"MEPS H131 CODEBOOK")
 final=prune_list(final,"MEPS HC-150")
 final=prune_list(final,"MEPS HC-158")
 final=prune_list(final,"MEPS H153 CODEBOOK")
 final=prune_list(final,"2012 FULL YEAR CONSOLIDATED DATA CODEBOOK")
 final=prune_list(final,"2013 FULL YEAR CONSOLIDATED DATA CODEBOOK")
 final=prune_list(final,"2013 FULL YEAR POPULATION CHARACTERISTICS CODEBOOK")
 final=prune_list(final,"2012 MEPS MEDICAL CONDITIONS FILE")
 final=prune_list(final,"2002-2009 RISK ADJUSTMENT SCORES FILE")
 final=prune_list(final,"EMPLOYMENT VARIABLE IMPUTATION FILE")
 final=prune_list(final,"2012 JOBS FILE CODEBOOK")
 final=prune_list(final,"2013 JOBS FILE CODEBOOK")
 final=prune_list(final,"2012 PERSON ROUND PLAN FILE")
 final=prune_list(final,"DATE:   August 25 2014")
 final=prune_list(final,"DATE:   August 25 2015")
 final=prune_list(final,"DATE: August 21 2014")
 final=prune_list(final,"DATE: August 4 2014")
 final=prune_list(final,"DATE:     March 6 2015")
 final=prune_list(final,"DATE: December 15 2014")
 final=prune_list(final,"DATE:    April 10 2013")
 final=prune_list(final,"DATE:   August 12 2014")
 final=prune_list(final,"DATE: February 13 2015")
 done=final
 with open(target,"w") as f:
  f.write("start,end,variable\n")
  for skip in range(0,len(final)-1,3):
   f.write("%s,"%final[skip])
   f.write("%s,"%final[skip+1])
   f.write("%s\n"%((final[skip+2]).lstrip()).rstrip())
 print("Done")
 temp=pd.read_csv(target)
 temp=temp.sort("start").copy()
 temp.index=list(range(len(temp)))
 return temp


Here's the definition of prune_list for reference:

In [15]:
def prune_list(thelist, theterm):
 while True:
  try:
   thelist.pop(thelist.index(theterm))
  except ValueError:
   break
 return thelist

Finally, here's the function that takes the MEPS dataset, and a dataframe containing the column dictionary, and writes the dataset out into a comma separated values file:

In [18]:
def write_table(data,header,target,short="No"):
 if short!="No":
  data=data[:10]
 data=prune_list(data,'')
 header=header.sort("start").copy()
 header.index=list(range(len(header)))
 with open(target,"w") as f:
  for element in header["variable"]:
   f.write("%s, "%element)
  f.write("\n")
 for row in data:
  u=[row[(header["start"].iloc[place]-1):(header["end"].iloc[place])] for place in header.index]
  with open(target,"a") as f:
   writer=csv.writer(f)
   writer.writerow(u)
  #print("Done with row %s"%data.index(row))
